In [17]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import seaborn as sns

#load in data for PA counties
PAcoviddat = pd.read_csv('PA-counties.csv')

PAdensitydat = pd.read_csv('PApopdensity.csv')

PAginidat = pd.read_csv('Ginidat.csv')

PAageover60 = pd.read_csv('PA-ageover60.csv')

# PAcoviddat["Density"] =[PAdensitydat['density']]

countarr = np.arange(1,68)

In [ ]:
#Plot of pop density vs deaths
plt.scatter(PAdensitydat['density'],PAcoviddat['deaths'], c = countarr)
plt.xlabel("Population Density")
plt.ylabel("Deaths")
cbar = plt.colorbar()
cbar.set_label('County')
plt.title('Population Density vs Deaths')
plt.show()

##plot of population density vs deaths excluding Philidelphia and Delaware (to get a better view at lower points trend)
plt.scatter(PAdensitydat['density'],PAcoviddat['deaths'], c = countarr)
plt.xlabel("Population Density")
plt.ylabel("Deaths")
cbar = plt.colorbar()
cbar.set_label('County')
plt.title('Population Density vs Deaths (Excluding Phili and Delaware)')
plt.xlim([0,800])
plt.show()

# ##plot of population over 60 vs deaths
# plt.scatter(PAageover60['Percent'],PAcoviddat['deaths'], c = countarr)
# plt.xlabel("Population over 60")
# plt.ylabel("Deaths")
# cbar = plt.colorbar()
# cbar.set_label('County')
# plt.title('Population Age over 60 vs Deaths')
# plt.show()


#Plot of gini index vs deaths
plt.scatter(PAginidat['Gini'],PAcoviddat['deaths'], c = countarr)
plt.xlabel("Gini Index")
plt.ylabel("Deaths")
cbar = plt.colorbar()
cbar.set_label('County')
plt.title('Gini Index vs Deaths')
plt.show()

#plot gini index vs population density
plt.scatter(PAginidat['Gini'],PAdensitydat['density'], c = countarr)
plt.xlabel("Gini Index")
plt.ylabel("Population Density")
cbar = plt.colorbar()
cbar.set_label('County')
plt.title('Gini Index vs Population Density')
plt.show()

#plot of gini index vs population density excluding Philidelphia and Delaware (to get a better view at lower points trend)
plt.scatter(PAginidat['Gini'],PAdensitydat['density'], c = countarr)
plt.xlabel("Gini Index")
plt.ylabel("Population Density")
cbar = plt.colorbar()
cbar.set_label('County')
plt.title('Gini Index vs Population Density (Excluding Phili and Delaware)')
plt.ylim([0,800])
plt.show()

# ##plot of population over 60 vs deaths
# plt.scatter(PAageover60['Percent'],PAginidat['Gini'], c = countarr)
# plt.xlabel("Population over 60")
# plt.ylabel("Gini Index")
# cbar = plt.colorbar()
# cbar.set_label('County')
# plt.title('Population Age over 60 vs Deaths')
# plt.show()

Since Pennsylvania is a large state with 67 counties, there are more than enough data points in order to conduct analysis through t/u-tests. Population density and gini index does not seem to show much dependence, at most there may be a slight upward trend. Although, some of the higher gini indexs are at a sub 100 population density.

In [18]:
#Are people in counties with higher population densities in PA more likely to have experienced fatal outcomes?

#Histogram of density to check if distribution is normal
# plt.hist(PAdensitydat['density'],100);
# plt.xlim([0,300])
# plt.title('histogram')
# plt.xlabel('Population Density');

#sets up a new column in the covid data with a value of 0 if the deaths is on the low end of the pop density
# and a 1 if it is on the high end
avgdensity = np.average(PAdensitydat['density'])
PAcoviddat["D H/L"] =[1 if x> avgdensity else 0 for x in PAdensitydat['density']]

print(PAcoviddat)
print(PAdensitydat)

          date        county         state   fips   cases  deaths  D H/L
0   10/15/2022         Adams  Pennsylvania  42001   29031     395      0
1   10/15/2022     Allegheny  Pennsylvania  42003  320559    3567      1
2   10/15/2022     Armstrong  Pennsylvania  42005   17323     372      0
3   10/15/2022        Beaver  Pennsylvania  42007   46424     781      0
4   10/15/2022       Bedford  Pennsylvania  42009   12241     283      0
..         ...           ...           ...    ...     ...     ...    ...
62  10/15/2022    Washington  Pennsylvania  42125   60439     691      0
63  10/15/2022         Wayne  Pennsylvania  42127   12129     183      0
64  10/15/2022  Westmoreland  Pennsylvania  42129   93270    1470      0
65  10/15/2022       Wyoming  Pennsylvania  42131    6076     115      0
66  10/15/2022          York  Pennsylvania  42133  137108    1582      1

[67 rows x 7 columns]
                 County         State     density       created_user  \
0          Adams County  Penn

In [19]:
#Are people in counties with higher income inequality (gini index) in PA more likely to have experienced fatal outcomes?

#sets up a new column in the covid data with a value of 0 if the deaths is on the low end of the gini index
# and a 1 if it is on the high end
avggini = np.average(PAginidat['Gini'])
PAcoviddat["G H/L"] =[1 if x> avggini else 0 for x in PAginidat['Gini']]

print(PAcoviddat)
print(PAdensitydat)

          date        county         state   fips   cases  deaths  D H/L  \
0   10/15/2022         Adams  Pennsylvania  42001   29031     395      0   
1   10/15/2022     Allegheny  Pennsylvania  42003  320559    3567      1   
2   10/15/2022     Armstrong  Pennsylvania  42005   17323     372      0   
3   10/15/2022        Beaver  Pennsylvania  42007   46424     781      0   
4   10/15/2022       Bedford  Pennsylvania  42009   12241     283      0   
..         ...           ...           ...    ...     ...     ...    ...   
62  10/15/2022    Washington  Pennsylvania  42125   60439     691      0   
63  10/15/2022         Wayne  Pennsylvania  42127   12129     183      0   
64  10/15/2022  Westmoreland  Pennsylvania  42129   93270    1470      0   
65  10/15/2022       Wyoming  Pennsylvania  42131    6076     115      0   
66  10/15/2022          York  Pennsylvania  42133  137108    1582      1   

    G H/L  
0       0  
1       1  
2       0  
3       0  
4       0  
..    ...  
62 